In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from utils import MyDataset

## Load model and test data

In [75]:
model = torch.load('./model/2999.pkl')
test_set = MyDataset("./dataset/demo/", "./dataset/demo/")
test_loader  = DataLoader(test_set, batch_size=1, 
                            shuffle=False, drop_last=True)

(263, 480, 480, 1)
(263, 480, 480, 1)


## Depth prediction visualization

In [77]:
%matplotlib inline
import cv2
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
index = 0
predictions = []
for origin_inputs, labels in test_loader:
    inputs = origin_inputs.permute(0,3,1,2).float()
    labels = labels.permute(0,3,1,2).float()

    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    predict = outputs.permute(0,2,3,1).data.cpu().numpy()

    predictions.append(predict[0])

    if index==0:
        plt.figure()

        plt.subplot(1,2,1)
        plt.imshow(origin_inputs[0])

        plt.subplot(1,2,2)
        plt.imshow(predict[0])

#     plt.savefig("./prediction/"+str(index)+".png")
    index+=1
np.save("./results/demo.npy", predictions)

d:\HKUST-Lab\Research\7-TorchSLAM\TorchSLAM\utils.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(self._X[data_index])
d:\HKUST-Lab\Research\7-TorchSLAM\TorchSLAM\utils.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_tensor = torch.tensor(self._Y[data_index])


## Mapping

In [96]:
import cv2
import numpy as np

alpha = 0.8

def grid_mapping(map, frame, index):
    map[index:index+frame.shape[0],0:frame.shape[1]] = alpha*map[index:index+frame.shape[0],0:frame.shape[1]] + (1-alpha)*frame
    return map

def localization(frame1, frame2):
    step = 0
    min_residual = np.sum(np.abs(frame1-frame2))
    for i in range(60):
        residual = np.sum(np.abs(frame1[i:,:]-frame2[:480-i,:]))
        if residual<=min_residual:
            step = i
            min_residual = residual
    return step

frames = np.load("./results/demo.npy")
map = np.zeros((1480, 480))
step = 0
pointer = 0

for i, frame in enumerate(frames):
    frame = frame[..., 0]
    if i==0:
        map[:frame.shape[0],:frame.shape[1]] = frame
    else:
        step = localization(last_frame, frame)
        map = grid_mapping(map, frame, pointer)
    pointer = step+pointer
    
    last_frame = frame

ran = np.max(map)-np.min(map)
map = (map-np.min(map)) / ran
map = 255*map

cv2.imwrite("../results/map.jpg", map)

True